Взять датасет из google диска: https://drive.google.com/file/d/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_' -O data.zip

--2022-04-21 08:21:08--  https://drive.google.com/uc?export=download&id=1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_
Resolving drive.google.com (drive.google.com)... 142.250.157.100, 142.250.157.138, 142.250.157.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.157.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7vc8gqadeqenucp1dhk542nnbvsthdq5/1650529200000/14904333240138417226/*/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_?e=download [following]
--2022-04-21 08:21:09--  https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7vc8gqadeqenucp1dhk542nnbvsthdq5/1650529200000/14904333240138417226/*/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_?e=download
Resolving doc-04-c0-docs.googleusercontent.com (doc-04-c0-docs.googleusercontent.com)... 108.177.97.132, 2404:6800:4008:c00::84
Connecting to doc-04-c0-docs.googleusercontent.com (doc-04-c0-

In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: marketing description.txt  
  inflating: marketing_campaign.csv  
  inflating: subscribers.csv         
  inflating: users.csv               


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats import proportion

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data = pd.read_csv('marketing_campaign.csv')
data.head(3)

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             10037 non-null  object
 1   date_served         10021 non-null  object
 2   marketing_channel   10022 non-null  object
 3   variant             10037 non-null  object
 4   language_displayed  10037 non-null  object
 5   converted           10022 non-null  object
dtypes: object(6)
memory usage: 470.6+ KB


In [ ]:
data.loc[data['converted'].isna(), 'converted'] = False

In [ ]:
a = 0.05 # Зафиксируем уровень значимости  α
k1 = data[data['variant']=='control']['converted'].sum()
n1 = data[data['variant']=='control'].shape[0]
k2 = data[data['variant']=='personalization']['converted'].sum()
n2 = data[data['variant']=='personalization'].shape[0]

k1, n1, k2, n2

(371, 5091, 705, 4946)

In [ ]:
grouped = data.pivot_table(values='converted', index='variant', aggfunc=['sum', 'count'])
grouped

,sum,count
,converted,converted
variant,,
control,371,5091
personalization,705,4946


_В тесте (personalization) конверсия (converted) больше чем в тесте (control)_

У нас 2 гипотезы:
- "нулевая" (H0) - наблюдаемые различия незначимы
- "альтернативная" (H1) - различия значимы

Посчитаем z-критерий

In [ ]:
z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), 
                                                 np.array([n1, n2]))
if z_pvalue <= a:
    print("Мы можем отвергнуть нулевую гипотезу!")
else:
    print("Нам не удалось отвергнуть нулевую гипотезу")
print(f'Результаты таковы z_score={z_score:.3f} pvalue={z_pvalue:.3f}')

Мы можем отвергнуть нулевую гипотезу!
Результаты таковы z_score=-11.279 pvalue=0.000


Посчитаем непараметрический критерий (хи-квадрат)

In [ ]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), 
                                                        np.array([n1, n2]))
if pvalue <= a:
    print("Мы можем отвергнуть нулевую гипотезу!")
else:
    print("Нам не удалось отвергнуть нулевую гипотезу")
print(f'Результаты таковы chisq={chisq:.3f} pvalue={pvalue:.3f}')

Мы можем отвергнуть нулевую гипотезу!
Результаты таковы chisq=127.213 pvalue=0.000


###_Вывод: разница в значениях конверсии (converted) в тестах (control/personalization) статистически значима._